In [1]:
import numpy as np
import os
import zipfile
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from flask import Flask, request, render_template_string
from werkzeug.utils import secure_filename
from PIL import Image
import io
import nest_asyncio
from IPython.display import display, HTML
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# ================== DATASET SETUP ==================
dataset_path = r'C:\Projects\Data_Science\QR_Code_Fraud_Detection'  # Raw string for Windows path
if not os.path.exists(dataset_path):
    raise ValueError(f"""
    Dataset directory not found at: {dataset_path}
    Please ensure you have:
    1. A 'dataset' folder in your working directory
    2. Subfolders 'first_print' and 'second_print' containing respective images
    """)

In [3]:
# ================== MODEL TRAINING ==================
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
img_size = (128, 128)

# Training generator (shuffled)
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=32,
    class_mode='binary',
    subset='training',
    classes=['first_print', 'second_print'],
    shuffle=True
)

Found 160 images belonging to 2 classes.


In [4]:
# Validation generator (not shuffled)
val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=32,
    class_mode='binary',
    subset='validation',
    classes=['first_print', 'second_print'],
    shuffle=False  # Critical for correct metric calculation
)


Found 40 images belonging to 2 classes.


In [5]:
# Build CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

C:\Users\prathamesh\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Train model
print("\nTraining model...")
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)



Training model...


C:\Users\prathamesh\AppData\Local\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.5576 - loss: 1.4589 - val_accuracy: 0.5000 - val_loss: 0.7154
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5933 - loss: 0.7090 - val_accuracy: 0.9250 - val_loss: 0.6558
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.8685 - loss: 0.6485 - val_accuracy: 0.5000 - val_loss: 0.6357
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5580 - loss: 0.6307 - val_accuracy: 0.9250 - val_loss: 0.5475
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9102 - loss: 0.5235 - val_accuracy: 0.8250 - val_loss: 0.4946
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.8351 - loss: 0.4211 - val_accuracy: 0.9000 - val_loss: 0.3756
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.8858 - loss: 0.3666 - val_accuracy: 0.9500 - val_loss: 0.2942
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9434 - loss: 0.2743 - val_accuracy: 0.8500 - val_loss: 0.3923
Epoch 9/10
5/5 

In [7]:
# ================== MODEL EVALUATION ==================
# Generate predictions for validation set
val_generator.reset()
y_pred = model.predict(val_generator)
y_pred_binary = (y_pred > 0.5).astype(int).flatten()
y_true = val_generator.classes

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred_binary)
precision = precision_score(y_true, y_pred_binary)
recall = recall_score(y_true, y_pred_binary)
f1 = f1_score(y_true, y_pred_binary)

print("\n\n=== Validation Metrics ===")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}\n")

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 202ms/step


=== Validation Metrics ===
Accuracy: 0.9250
Precision: 0.9048
Recall: 0.9500
F1-Score: 0.9268



In [ ]:

# ================== FLASK APPLICATION ==================
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'

HTML_TEMPLATE = '''
<!doctype html>
<html>
  <head><title>QR Code Authenticator</title></head>
  <body>
    <h1>Upload QR Code</h1>
    <form method="post" enctype="multipart/form-data">
      <input type="file" name="file" accept="image/*">
      <input type="submit" value="Verify">
    </form>
    {% if result %}
    <h2>Result: {{ result }}</h2>
    {% endif %}
  </body>
</html>
'''

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return 'No file uploaded'
        file = request.files['file']
        if file.filename == '':
            return 'No selected file'
        
        # Preprocess image
        img = Image.open(file.stream).convert('RGB')
        img = img.resize((128, 128))
        img_array = np.array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        
        # Make prediction
        prediction = model.predict(img_array)
        result = "Original" if prediction[0][0] < 0.5 else "Counterfeit"
        return render_template_string(HTML_TEMPLATE, result=result)
    
    return render_template_string(HTML_TEMPLATE)

# Allow Flask to run in Jupyter
nest_asyncio.apply()
print("\nLaunching web interface...")
display(HTML(f'<a href="http://localhost:3000" target="_blank">Open QR Code Authenticator</a>'))
app.run(port=3000, threaded=True)


Launching web interface...


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3000
Press CTRL+C to quit
